# PyTorch: Evaluating

https://github.com/pytorch/examples/tree/master/mnist

In [2]:
import os
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np

In [3]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 1
no_cuda = False
log_interval = 100

# Model

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Preprocess

In [5]:
datasets.MNIST('../dataset', train=True, download=True,
               transform=transforms.Compose([
                   transforms.ToTensor(),
                   transforms.Normalize((0.1307,), (0.3081,))
               ]))

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../dataset\MNIST\raw\train-images-idx3-ubyte.gz to ../dataset\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../dataset\MNIST\raw\train-labels-idx1-ubyte.gz to ../dataset\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../dataset\MNIST\raw\t10k-images-idx3-ubyte.gz to ../dataset\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../dataset\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../dataset\MNIST\raw



Dataset MNIST
    Number of datapoints: 60000
    Root location: ../dataset
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [6]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}



train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)




test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data\MNIST\raw\train-images-idx3-ubyte.gz to ../data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data\MNIST\raw\train-labels-idx1-ubyte.gz to ../data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data\MNIST\raw\t10k-images-idx3-ubyte.gz to ../data\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../data\MNIST\raw



# Optimization

In [7]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [8]:
params = list(model.parameters())

In [9]:
for i in range(8):
    print(params[i].size())

torch.Size([20, 1, 5, 5])
torch.Size([20])
torch.Size([50, 20, 5, 5])
torch.Size([50])
torch.Size([500, 800])
torch.Size([500])
torch.Size([10, 500])
torch.Size([10])


# Training

In [10]:
for epoch in range(1, epochs + 1):
    # Train Mode
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()  # backpropagation 계산하기 전에 0으로 기울기 계산
        output = model(data)
        loss = F.nll_loss(output, target)  # https://pytorch.org/docs/stable/nn.html#nll-loss
        loss.backward()  # 계산한 기울기를 
        optimizer.step()  

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.299172
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.233125
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.125046
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.954126
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.746479
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.243096
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.026327
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.617802
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.717742
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.411956


# Evaluation

- 앞에서 model.train() 모드로 변한 것처럼 평가 할 때는 model.eval()로 설정
    - Batch Normalization이나 Drop Out 같은 Layer들을 잠금

In [11]:
model.eval()

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

- autograd engine, 즉 backpropagatin이나 gradient 계산 등을 꺼서 memory usage를 줄이고 속도를 높임

In [13]:
test_loss = 0
correct = 0

with torch.no_grad():
    data,target = next(iter(test_loader))
    data,target = data.to(device), target.to(device)
    output = model(data)
    
    test_loss += F.nll_loss(output, target, reduction='sum').item()
    
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(target.view_as(pred)).sum().item()
    
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    (
    print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset))))

In [16]:
pred

tensor([[4],
        [4],
        [2],
        [1],
        [9],
        [1],
        [8],
        [2],
        [4],
        [8],
        [7],
        [4],
        [4],
        [7],
        [7],
        [8],
        [4],
        [4],
        [1],
        [3],
        [5],
        [4],
        [6],
        [6],
        [3],
        [8],
        [7],
        [1],
        [4],
        [9],
        [1],
        [9],
        [2],
        [0],
        [3],
        [0],
        [7],
        [7],
        [2],
        [0],
        [5],
        [3],
        [9],
        [9],
        [5],
        [8],
        [7],
        [9],
        [5],
        [1],
        [4],
        [1],
        [0],
        [6],
        [9],
        [1],
        [7],
        [3],
        [1],
        [3],
        [6],
        [4],
        [6],
        [6]])

In [14]:
test_loss

32.410823822021484

In [15]:
correct

55

In [17]:
pred.shape

torch.Size([64, 1])

In [18]:
output.shape

torch.Size([64, 10])

In [19]:
target

tensor([4, 9, 2, 1, 4, 1, 8, 2, 4, 8, 9, 4, 4, 7, 7, 8, 4, 4, 1, 3, 5, 5, 6, 6,
        3, 8, 7, 1, 4, 7, 1, 9, 7, 0, 3, 0, 7, 7, 2, 0, 5, 3, 9, 9, 5, 8, 7, 4,
        5, 1, 4, 1, 0, 6, 9, 1, 7, 3, 1, 3, 2, 4, 6, 2])

In [20]:
target.shape

torch.Size([64])

In [21]:
target.view_as(pred).shape#shape 맞춰줌

torch.Size([64, 1])

In [22]:
pred.eq(target.view_as(pred))

tensor([[ True],
        [False],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True

In [23]:
pred.eq(target.view_as(pred)).sum()

tensor(55)

In [24]:
pred.eq(target.view_as(pred)).sum().item()/64

0.859375

In [25]:
test_loss /= len(test_loader.dataset)

In [26]:
test_loss #평균값

0.0032410823822021486

In [28]:
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
test_loss /= len(test_loader.dataset)
(
print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset))))


Test set: Average Loss: 0.4822, Accuracy: 8670/10000 (87%)

